In [1]:
# Imports
import osmnx as ox
import matplotlib
import mapclassify
import folium
import pandas as pd
import geopandas as gpd
import h3pandas
import h3

from shapely.geometry import MultiPolygon # Buildings
import time
import re # for population feature

from pathlib import Path
import gc

In [2]:
# Print the versions of pkgs
print("osmnx", ox.__version__)
print("pandas", pd.__version__)
print("geopandas", gpd.__version__)
print("folium", folium.__version__)
print("h3pandas", h3pandas.__version__)

osmnx 1.5.1
pandas 2.0.3
geopandas 0.13.2
folium 0.14.0
h3pandas 0.2.4


In [3]:
# Get the current working directory
cwd = Path.cwd()

# Create a paths to the features and temp directories
RAW_DATA_PATH = cwd / "Data"
TEMP_DATA_PATH = cwd / "Temp"
FEATURE_PATH = cwd / "Features"
LANDUSE_PATH = FEATURE_PATH / "landuse"
POI_PATH = FEATURE_PATH / "poi"
DIST_PATH = FEATURE_PATH / "distance"

# Create constant for osmnx and crs
PLACE = "Malmö kommun"
CRS = 3006
WGS = 4326

In [4]:
# Create dirs for paths
RAW_DATA_PATH.mkdir(parents=True, exist_ok=True)
TEMP_DATA_PATH.mkdir(parents=True, exist_ok=True)
FEATURE_PATH.mkdir(parents=True, exist_ok=True)
LANDUSE_PATH.mkdir(parents=True, exist_ok=True)
POI_PATH.mkdir(parents=True, exist_ok=True)
DIST_PATH.mkdir(parents=True, exist_ok=True)

## RegSo

### Upload RegSo

In [5]:
# Upload raw data frame with regso information
regso_city = gpd.read_parquet(f"{RAW_DATA_PATH}/malmo_regso.parquet")[['regsokod', 'regso','kommunnamn', 'geometry']].to_crs(WGS)
print(f"Shape of regso_city: {regso_city.shape}")

Shape of regso_city: (91, 4)


In [6]:
# Print head of data frame
regso_city.head(2)

,regsokod,regso,kommunnamn,geometry
0,1280R003,RegSO-Annelund-Lönngården,Malmö,"POLYGON ((13.03129 55.58991, 13.03111 55.58954..."
1,1280R044,RegSO-Limhamns hamnområde,Malmö,"POLYGON ((12.94297 55.59082, 12.94192 55.59022..."


In [7]:
# # Plot the polygons on the map
# regso_city.to_crs(4326).explore()

As we can see in the map above, the geographical representation of areas includes water bodies (rivers, canals, lakes, etc.) for our future analysis it is necessary to exclude some objects from the data.

For this purpose, we will use geodata of coastline of Sweden.<br><br>

You can find Geopackages file EEA coastline Polygon on [European Environment Agency](https://www.eea.europa.eu/data-and-maps/data/eea-coastline-for-analysis-2/gis-data/eea-coastline-polygon)


In [8]:
# # Parquet file with coastline of Sweden CRS: EPSG:3006
# coastline = gpd.read_parquet(
#     "Data/SE_Coastline.parquet"
# )
# coastline.columns = coastline.columns.str.lower()
# coastline_unary = coastline.geometry.unary_union
# # Create the GeoDataFrame for the bounding box
# se_bbox = gpd.GeoDataFrame({'geometry': [coastline_unary]}, crs=coastline.crs)

In [9]:
# # Slice regso_city with coastline
# regso_city = gpd.overlay(regso_city.to_crs(CRS), se_bbox, how='intersection')

In [10]:
# # Plot the polygons on the map
# regso_city.to_crs(4326).explore()

In [11]:
# del coastline
# del coastline_unary

### RegSo transformation

- Processing data by districts (RegSo) of the city of Malmö `malmo_regso.parquet`
    - Transforming the dataset `malmo_regso.parquet` with district (RegSo) geopolygons into a dataset with H3 objects and district features (resolution = 9).

In [12]:
# Adding h3 to RegSo
h3_regso_city = (regso_city.to_crs(WGS).h3.polyfill(9, explode=True)
                      .dropna().set_index('h3_polyfill', drop=True)).h3.h3_to_geo_boundary()
print(f"Shape of h3_regso_city: {h3_regso_city.shape}")

Shape of h3_regso_city: (2028, 4)


In [13]:
# # Plot the polygons on the map
# h3_regso_city.to_crs(4326).explore()

## Target (outgoing trips)

- Loading a dataset containing information on the number of outgoing trips from each h3 polygon for which data is available on the observation date. (01/06/23-02/06/23)

- Upload targets h3
- Add features to `h3_regso_city` is h3 area include outgoing trips or not ('targets_intersects')

### Upload target

In [14]:
# Upload raw data frame with targets
h3_target = pd.read_csv(f'{RAW_DATA_PATH}/malmo_outgoing_trips.csv').set_index('hex')
print(f"Shape of h3_target: {h3_target.shape}")

Shape of h3_target: (468, 1)


## Combine RegSo and target

- Add a feature indicating whether rental points are present in the district (whether there are trips from this area) `targets_intersects`.
    - This is necessary for analyzing the feature space of districts where infrastructure is absent but rental points may be established in the futureure.

In [15]:
# Create new column with values of outgoing_trips and boolean values

if "outgoing_trips" in(h3_regso_city.columns):
    pass
else:
    h3_regso_city = h3_regso_city.join(h3_target)
    regsokod_intersects = ((h3_regso_city.groupby("regsokod")["outgoing_trips"]
                            .sum()) > 0).to_frame().reset_index().rename(columns={"outgoing_trips": "targets_intersects"})
    h3_regso_city = h3_regso_city.reset_index().merge(regsokod_intersects, on="regsokod").set_index("h3_polyfill")
    h3_regso_city.fillna(0.0, inplace=True)
    print(f"Shape of h3_regso_city: {h3_regso_city.shape}")

Shape of h3_regso_city: (2028, 6)


### Export h3_regso_city

In [16]:
# Export h3_regso_city to temp dir
h3_regso_city.to_parquet(f'{TEMP_DATA_PATH}/h3_regso_city.parquet')

### Export h3_features

In [17]:
# Create df with features and export
h3_features = h3_regso_city.copy()
h3_features.to_parquet(f'{FEATURE_PATH}/h3_features.parquet')

## Landuse features

- 

In [18]:
# Create dict with features as keys and osm tags as values
landuse_osm = {
    "f0": {"landuse":"residential"},
    "f01": {"landuse":["allotments", "farmland", "village_green"]},
    "f03": {"landuse":["commercial", "retail"]},
    "f04": {"landuse":["industrial", "quarry"]},
    "f05": {"landuse":["reservoir"], "natural":["water","river"], "water":["river"]},
    "f06": {"landuse":"landfill"},
    "f07": {"landuse":"garages"},
    "f08": {"landuse":"railway"},
    "f12": {"landuse":"military"},
    "f13": {"landuse":"basin"},
    "f14": {"landuse":["construction", "brownfield", "proposed", "greenfield"]},
    "f15": {"landuse":["farmyard", "animal_keeping", "vineyard", "orchard", "shrubs", "greenhouse_horticulture"]},
    "f16": {"landuse":["religious", "cemetery"]},
    "f17": {"landuse":"education"},
    "f18": {"landuse":["forest", "meadow", "flowerbed", "recreation ground"], #, "grass"
            "leisure":["park", "garden"], "natural":["wood"]}
}

<div style="background-color:lightgreen; padding:10px;">
Load data with features from OSM using osmnx.


- Internet connection is required.
  
</div>.

In [ ]:
# Extract (upload) from osm data for features
for index, feature in enumerate(landuse_osm):
    values_list = list(landuse_osm.values())
    tags = values_list[index]
    if len(tags) == 1:
        tag = list(tags)[0]
        try:
            gdf = ((ox.features_from_place(PLACE, tags).reset_index())
                   [["osmid", str(tag), "geometry"]]).to_crs(WGS)
            gdf.to_parquet(f"{LANDUSE_PATH}/{feature}.parquet")
            print(f"{feature}")
        except Exception as e:
            print(f"{feature} {e}")
    else:
        gdf_list = []
        for item in range(len(tags)):
            tag = list(tags)[item]
            dict_item = {list(tags.keys())[item]: list(tags.values())[item]}
            try:
                gdf = ((ox.features_from_place(PLACE, dict_item).reset_index())
                   [["osmid", str(tag), "geometry"]]).to_crs(WGS).rename(columns={str(tag):"landuse"})
                gdf_list.append(gdf)
            except Exception as e:
                print(f"{feature} {e}")
        result = pd.concat(gdf_list, ignore_index=True, copy=False)

        # Export files with data of features
        result.to_parquet(f"{LANDUSE_PATH}/{feature}.parquet")
        print(f"{feature}")
        
# Delete the temporal variables
del gdf_list
del gdf

In [18]:
# Clear f18 feature
f18 = gpd.read_parquet(f"{LANDUSE_PATH}/f18.parquet")
print("Shape before: ", f18.shape)
f18["area"] = f18.to_crs(CRS)["geometry"].area
f18 = f18.query("area > 10000").drop(columns=["area"])

print("Shape after: ", f18.shape)
# Update f18 file
f18.to_parquet(f"{LANDUSE_PATH}/f18.parquet")

Shape before:  (312, 3)
Shape after:  (312, 3)


### Create h3_features_landuse

- Extract information from landuse features files and aply to h3
- For landuse features extract information is h3 overlaps the polygons of landuse

In [19]:
# Create a list of Paths for landuse features files
f_files = sorted(list(LANDUSE_PATH.glob("f*.parquet")))

In [20]:
# Change crs of gdf
h3_ = h3_regso_city[["geometry"]].to_crs(CRS)

In [21]:
# Start time
start_time = time.time()

# Generate features 
for _ in range(len(f_files)):
    temp_file = gpd.read_parquet(f"{f_files[_]}")
    column_name = f_files[_].stem
    union = temp_file.to_crs(CRS)["geometry"].unary_union
    h3_[str(column_name)] = (h3_["geometry"].intersects(union).astype('int')) #1_overlaps #3_intersects
    print(f"{column_name} Done")

# End time
end_time = time.time()

# Calculate overall time taken in seconds
overall_time_seconds = end_time - start_time

# Convert overall time to minutes and seconds
overall_minutes, overall_seconds = divmod(overall_time_seconds, 60)

print("Overall time taken: {} minutes and {:.2f} seconds".format(int(overall_minutes), overall_seconds))

f0 Done
f01 Done
f03 Done
f04 Done
f05 Done
f07 Done
f08 Done
f12 Done
f13 Done
f14 Done
f15 Done
f16 Done
f17 Done
f18 Done
Overall time taken: 2 minutes and 6.66 seconds


### Export h3_features_landuse

In [22]:
# Export gdf with landuse features
h3_.to_parquet(f'{TEMP_DATA_PATH}/h3_features_landuse.parquet')

### Update h3_features

In [23]:
# # Add features_landuse to h3_features
# h3_features = h3_regso_city.join(h3_.drop(columns="geometry"))
# h3_features.to_parquet(f'{FEATURE_PATH}/h3_features.parquet')

## K-ring features

- 

In [24]:
# Create dict with features as keys and osm tags as values
kring_osm = {
    "f20": {"amenity": ["restaurant", "pub", "bar", "cafe", "fast_food", "food_court"],
            "shop": ["general", "supermarket", "grocery"]},
    "f21": {"leisure": ["sports_centre", "sports_hall", "stadium", "track", "pitch", ]}, #"swimming_pool"
    "f22": {"tourism":["hotel", "guest_house", "apartment"]},
    
    "f25": {"healthcare":["clinic", "hospital"], "amenity": ["clinic", "hospital", "retirement_home"]},
    "f26": {"amenity": ["school"]},
    "f27": {"amenity": ["kindergarten"]},
    
    "f28": {"tourism": ["museum", "gallery"], "amenity": ["arts_centre", "theatre"]},
    "f29": {"amenity": ["library"]},
    "f30": {"amenity": ["courthouse", "townhall"], "office": ["government"]},
    "f31": {"amenity": ["fountain"], "leisure": ["water_park"], "tourism":["attraction", "theme_park", 'viewpoint']}
}

<div style="background-color:lightgreen; padding:10px;">

Load data with features from OSM using osmnx.


- Internet connection is required.
</div>

In [28]:
# Extract (upload) from osm data for features
for index, feature in enumerate(kring_osm):
    values_list = list(kring_osm.values())
    tags = values_list[index]
    if len(tags) == 1:
        tag = list(tags)[0]
        try:
            gdf = ((ox.features_from_place(PLACE, tags).reset_index())
                   [["osmid", str(tag), "geometry"]]).to_crs(CRS)
            gdf['centroid'] = gdf.geometry.to_crs(CRS).centroid
            gdf = (gdf.drop(columns=['geometry'])
                   .rename(columns={str(tag):"type", "centroid":"geometry"})
                   .set_geometry('geometry'))
            gdf.to_parquet(f"{POI_PATH}/{feature}.parquet")
            print(f"{feature}")
        except Exception as e:
            print(f"{feature} {e}")
    else:
        gdf_list = []
        for item in range(len(tags)):
            tag = list(tags)[item]
            dict_item = {list(tags.keys())[item]: list(tags.values())[item]}
            try:
                gdf = ((ox.features_from_place(PLACE, dict_item).reset_index())
                   [["osmid", str(tag), "geometry"]]).to_crs(CRS).rename(columns={str(tag):"type"})
                gdf['centroid'] = gdf.geometry.to_crs(CRS).centroid
                gdf = (gdf.drop(columns=['geometry'])
                       .rename(columns={str(tag):"type", "centroid":"geometry"})
                       .set_geometry('geometry'))
                gdf_list.append(gdf)
            except Exception as e:
                print(f"{feature} {e}")
        result = pd.concat(gdf_list, ignore_index=True, copy=False)

        # Export files with data of features
        result.to_parquet(f"{POI_PATH}/{feature}.parquet")
        print(f"{feature}")

f20
f21
f22
f25
f26
f27
f28
f29
f30
f31


### Calculating POI for h3 neighbors

In [25]:
# Define the function to sjoin POI to h3
def poi_to_h3(h3_gdf, poi_gdf):
    h3_poi = ((gpd.sjoin(h3_gdf.to_crs(CRS), poi_gdf.to_crs(CRS)))
        .reset_index()
        .groupby('h3_polyfill')['index_right'].count()).to_frame()
    return h3_poi

In [26]:
# Define the function to calculate the sum of values for H3 neighbors


def calculate_neighbor_sum(x, data_frame, value_column):
    hex_neighbors = h3.k_ring(x, 1)  # Get the H3 neighbors within the ring of radius 1
    data_frame = data_frame.reset_index()
    neighbor_sum = data_frame.loc[data_frame['h3_polyfill'].isin(hex_neighbors)][value_column].sum()
    return neighbor_sum

In [27]:
# Create a list of Paths for poi features files
f_files = sorted(list(POI_PATH.glob("f*.parquet")))

In [28]:
# Count all POI for each h3 neighbors
h3_poi = (h3_regso_city[["geometry"]]).reset_index()
for _ in range(len(f_files)):
    temp_file = gpd.read_parquet(f"{f_files[_]}")
    column_name = f_files[_].stem
    h3_temp = poi_to_h3(h3_poi, temp_file[["geometry"]])
    h3_poi[str(column_name)] = h3_poi["h3_polyfill"].apply(lambda x: calculate_neighbor_sum(x, h3_temp, "index_right"))
print("Done")

Done


### Export h3_features_poi

In [29]:
# Export file
h3_poi.set_index('h3_polyfill', drop=True).to_parquet(f"{TEMP_DATA_PATH}/h3_features_poi.parquet")

### Update h3_features

In [30]:
# # Update file h3_features
# h3_features = h3_features.join(h3_poi.set_index('h3_polyfill', drop=True).drop(columns="geometry"))
# h3_features.to_parquet(f"{FEATURE_PATH}/h3_features.parquet")

## Distance features

In [31]:
# Create dict with features as keys and osm tags as values
dist_osm = {
    "f32": {"amenity": ["university"]}, #, "college"
    "f33": {"amenity": ["bus_station"], "highway": ["bus_stop"]},
    "f34": {"railway": ['station', 'subway_entrance']}, # 'tram_stop'
    "f35": {"highway":["cycleway"]},
    "f36": {"highway": ["residential", "tertiary"]},
    "f37": {"shop": ["mall"]}
}

<div style="background-color:lightgreen; padding:10px;">

Load data with features from OSM using osmnx.


- Internet connection is required.
</div>

In [36]:
# Extract (upload) from osm data for features
for index, feature in enumerate(dist_osm):
    values_list = list(dist_osm.values())
    tags = values_list[index]
    if len(tags) == 1:
        tag = list(tags)[0]
        try:
            gdf = ((ox.features_from_place(PLACE, tags).reset_index())
                   [["osmid", str(tag), "geometry"]]).to_crs(WGS)
            gdf.to_parquet(f"{DIST_PATH}/{feature}.parquet")
            print(f"{feature}")
        except Exception as e:
            print(f"{feature} {e}")
    else:
        gdf_list = []
        for item in range(len(tags)):
            tag = list(tags)[item]
            dict_item = {list(tags.keys())[item]: list(tags.values())[item]}
            try:
                gdf = ((ox.features_from_place(PLACE, dict_item).reset_index())
                   [["osmid", str(tag), "geometry"]]).to_crs(WGS).rename(columns={str(tag):"landuse"})
                gdf_list.append(gdf)
            except Exception as e:
                print(f"{feature} {e}")
        result = pd.concat(gdf_list, ignore_index=True, copy=False)

        # Export files with data of features
        result.to_parquet(f"{DIST_PATH}/{feature}.parquet")
        print(f"{feature}")

f32
f33
f34
f35
f36
f37


In [32]:
# Create a list of Paths for dist features files
f_files = sorted(list(DIST_PATH.glob("f*.parquet")))

In [33]:
# Calculate distance to near feature
h3_centroid = h3_regso_city[["geometry"]].copy()
h3_centroid["centroid"] = h3_centroid.geometry.to_crs(CRS).centroid
h3_centroid = (h3_centroid.set_geometry('centroid')).drop(columns="geometry")

for _ in range(len(f_files)):
    temp_file = gpd.read_parquet(f"{f_files[_]}").to_crs(CRS) #.explode() #(index_parts=True)
    temp_file["geom"] = temp_file.geometry
    column_name = f_files[_].stem
    temp_dist = (gpd.sjoin_nearest
                             (h3_centroid, temp_file,how='inner',distance_col=f'{column_name}'))
    
    temp_dist = (temp_dist[[f'{column_name}']].reset_index()
                     .drop_duplicates('h3_polyfill',keep='last',).set_index('h3_polyfill',drop=True))
    h3_centroid = h3_centroid.join(temp_dist)

h3_centroid.drop(columns="centroid", inplace=True)
print("Done")

Done


### Export h3_features_dist

In [34]:
# Export file
h3_centroid.to_parquet(f"{TEMP_DATA_PATH}/h3_features_dist.parquet")

### Update h3_features

In [35]:
# # Add features_dist to h3_features
# h3_features = h3_features.join(h3_centroid)
# h3_features.to_parquet(f'{FEATURE_PATH}/h3_features.parquet')

## Buildings features

<div style="background-color:lightgreen; padding:10px;">

Load data with features from OSM using osmnx.


- Internet connection is required.
</div>

In [ ]:
# Extract (upload) buildings feature from osm
dict_item = {"building": True}
buildings_feature = ((ox.features_from_place(PLACE, dict_item).reset_index())
                   [["osmid", "building", "geometry"]]).to_crs(CRS)

buildings_feature = buildings_feature[buildings_feature.geometry.geom_type != 'Point']
print("Done")
# Export file
buildings_feature.to_parquet(f"{TEMP_DATA_PATH}/h3_buildings.parquet")

### Calculating the area of buildings for h3 cells

In [36]:
# Create temp gdf
h3_regso_city_crs = h3_regso_city[["geometry"]].to_crs(CRS)
buildings_feature = gpd.read_parquet(f"{TEMP_DATA_PATH}/h3_buildings.parquet")
unary = buildings_feature.to_crs(CRS).unary_union
intersection_areas = []
# intersection_true = []

for index, row in h3_regso_city_crs.iterrows():
    # Perform the intersection with the MultiPolygon
    intersection = row['geometry'].intersection(unary)
    
    # Calculate the area of the intersection and append it to the list
    intersection_area = intersection.area
    intersection_areas.append(intersection_area)
    # intersection_ = int(intersection_area != 0)
    # intersection_true.append(intersection_)
    
# Add a new column 'intersection_area' to the gdf
h3_regso_city_crs['intersection_area'] = intersection_areas
# h3_regso_city_crs['f19_1'] = intersection_true
h3_regso_city_crs["f19"] = h3_regso_city_crs["intersection_area"] / 105332.5 # !!!
print("Done")

Done


### Export h3_features_buildings

In [37]:
# Export file
h3_regso_city_crs.to_parquet(f"{TEMP_DATA_PATH}/h3_features_buildings.parquet")

### Update h3_features

In [38]:
# # Update file
# h3_features = gpd.read_parquet(f"{FEATURE_PATH}/h3_features.parquet")
# h3_features = h3_features.join(h3_regso_city_crs[["f19"]])
# h3_features.to_parquet(f"{FEATURE_PATH}/h3_features.parquet")

In [39]:
# Create regso_build_area
build = h3_regso_city.join(h3_regso_city_crs[["f19"]])
regso_build_area = build[['regso', 'f19']].groupby('regso')['f19'].sum().to_frame()

## Population

- Number of persons living within a grid cell (mean)
- Data from Regso

In [40]:
# Upload raw data
pop_regso = pd.read_csv(f"{RAW_DATA_PATH}/regso_population_22.csv", encoding='latin-1')

In [41]:
# Create function for matching columns value


def extract_substring(row):
    string = row['region']  
    
    match = re.search(r'\((.*?)\)', string)
    
    if match:
        extracted_string = match.group(1)
        return extracted_string
    else:
        return "No match found."

#### NEW

In [42]:
# Update columns value
pop_regso['regso'] = pop_regso.apply(extract_substring, axis=1)
pop_regso = pop_regso[['regso','2022']]
pop_regso = pop_regso.merge(regso_build_area, on='regso')
pop_regso['mean_pop'] = (pop_regso['2022'] / pop_regso['f19'])
pop_regso.drop(columns=['2022','f19'], inplace=True)

In [43]:
# Merge popolation data with h3
h3_pop = h3_regso_city[["regso", "geometry"]]
h3_pop = (h3_pop.reset_index().merge(pop_regso, on='regso')).set_index("h3_polyfill").drop(columns="regso")

In [44]:
# Calculate population for h3
h3_pop = h3_pop.join(h3_regso_city_crs[["f19"]])
h3_pop["f38"] = h3_pop["mean_pop"] * h3_pop["f19"]

In [45]:
h3_pop.f38.sum()

355803.0

In [46]:
h3_pop.explore(tooltip="f38", popup=False, column="f38", cmap="plasma", style_kwds=dict(color="white", weight=0.2, fillOpacity=0.3))

### Export h3_features_pop

In [47]:
# Export file
h3_pop.to_parquet(f"{TEMP_DATA_PATH}/h3_features_pop.parquet")

### Update h3_features

In [25]:

# Add features_landuse to h3_features
h3_features = gpd.read_parquet(f'{TEMP_DATA_PATH}/h3_regso_city.parquet')

h3_features_landuse = gpd.read_parquet(f'{TEMP_DATA_PATH}/h3_features_landuse.parquet')
h3_features = h3_regso_city.join(h3_features_landuse.drop(columns="geometry"))

h3_features_poi = gpd.read_parquet(f"{TEMP_DATA_PATH}/h3_features_poi.parquet")
h3_features = h3_features.join(h3_features_poi.drop(columns="geometry"))

h3_features_dist = pd.read_parquet(f"{TEMP_DATA_PATH}/h3_features_dist.parquet")
h3_features = h3_features.join(h3_features_dist)

h3_features_buildings = gpd.read_parquet(f"{TEMP_DATA_PATH}/h3_features_buildings.parquet")
h3_features = h3_features.join(h3_features_buildings[["f19"]])

# Add features_dist to h3_features
h3_features_pop = gpd.read_parquet(f"{TEMP_DATA_PATH}/h3_features_pop.parquet")
h3_features = h3_features.join(h3_features_pop[["f38"]])
h3_features.sort_index().to_parquet(f'{FEATURE_PATH}/h3_features.parquet')

### Export the features dataset to Model dir

In [26]:
from datetime import datetime
parent_dir = cwd.parent
model_dir = parent_dir / "Model" / "Data"
model_dir.mkdir(parents=True, exist_ok=True)
# Format the date and time as a string
datetime_ = datetime.now().strftime("%d%m%y_%H%M%S")
h3_features.sort_index().to_parquet(f"{model_dir}/features_dataset_{datetime_}.parquet")